## Conventions:

- turn e player_id partono da 1
- il turno viene aggiornato non appena il giocatore di turno gioca le carte. Dunque si dubita sempre del giocatore del turno precedente, fintanto che il nuovo giocatore non gioca carte a sua volta

## General todoes

- implementare il check_input
- rendere il tutto meno spaghettato (es. alcuni assert sono in player anziché in game)
- assegnare il turno a chi vince, o al giocatore che ha iniziato la mano se tutti passano

--- 

- **safety checks** -> controlla che:
    - non puoi dubitare di un giocatore che ha passato
    - se dubiti e perdi peschi le carte
    - se tutti passano il pozzo viene resettato e rimani con le carte di prima
    - se rimani senza carte vinci (magari un random seed?)
    - se rimani senza carte e vieni ingiustamente dubitato vinci
    - se rimani senza carte ma vieni correttamente dubitato non vinci
    - rendi tutto eseguibile via terminal, e deve tutto terminare quando uno vince
    - assicurati che non si possano fare due doubt consecutivi (non ci sono conseguenze sulle carte, ma potrebbero essercene sulla gestione del turno)
    - molto spaghetti code come gestisco i player id e i turni, controlla qui se ci sono bug


# Cards 

In [1]:
# -*- coding: utf-8 -*-
"""
Contiene la classe Cards, per gestire il mazzo,
e la funzione safety_checks_Deck, per effettuare
alcuni controlli base sulla classe

@author: Eugenio
"""
import numpy as np
import random as random
import re
import time


class Cards:
    def __init__(self, num_players, jolly = 100):
        self.num_players = num_players
        self.to_draw = 0 
        self.deck = []
        self.pool = [] 
        self.jolly = jolly
        
        assert isinstance(jolly, int), "Il Jolly deve essere un numero"
        assert isinstance(num_players, int), "num_players non è un numero"
        
        self.create_deck()
    
    def create_deck(self):
        """ Crea un deck con 4 copie di ogni carta (una per seme)
            e 4 jolly, indicati per convenzione col numero 100 """
        
        self.deck = np.array([i for i in range(1,14)]*4 +[self.jolly]*4, dtype=int)
        if self.num_players > 5:
            self.deck = np.array[list(self.deck) + list(self.deck)]
        
        self.to_draw = len(self.deck)//self.num_players
        self.shuffle()
        
    def shuffle(self):
        random.shuffle(self.deck)
        
    def add_to_pool(self, cards):
        self.pool += list(cards)
        
    def reset_pool(self):
        self.pool = []
        print("Le carte sono state prese o rimosse dal pozzo")
        
                
    def draw(self):
        """ draw an equal amount of cards from the deck for each player.
            Return the cards extracted for one player, while removing
            them from the Deck.
            Note: requires that the Deck remains shuffled"""
        
        # safety check
        assert len(self.deck) >= self.to_draw, "There are not enough cards to draw!"
        
        # extracted cards
        #extracted = np.random.choice(self.deck, self.to_draw, replace = False)
        extracted = self.deck[:self.to_draw]
        
        # remove them from deck
        #self.deck = np.array([i for i in self.deck if i not in extracted])
        self.deck = self.deck[self.to_draw:]
        
        return extracted

# Player

todo:
- 

In [2]:
# -*- coding: utf-8 -*-
"""
Contiene la classe Player, per gestire il mazzo,
e la funzione safety_checks_Deck, per effettuare
alcuni controlli base sulla classe

@author: Eugenio
"""

import numpy as np
import random as random
import re
import time


class Player:
    def __init__(self, game, id_):
        self.hand = []
        self.game = game
        self.id_ = id_
        
        game.add_player(self)
                
    def add_to_hand(self, cards):
        # se la chiami __add_to_hand fai sì che non può essere chiamata se non da altre internal functions
        
        # volendo puoi sovrascrivere __iadd__ per poter operare comodamente con np arrays
        self.hand += list(cards)
        
    def whoami(self):
        print(f"sei il giocatore {self.id_}")
        
    def achitocca(self):
        print(f"it's the turn of player {self.game.turn}")
    
    def show_hand(self):
        print("Le tue carte sono: ", sorted(self.hand))
    
    def reset_hand(self):
        self.hand = []
        
    def vote_Start(self):
        self.game.vote_Start(self)
        
    def first_draw(self):
        print("pesco le carte iniziali")
        self.add_to_hand(self.game.first_draw())
              
    def declare_card(self, card_declared):
        
        game = self.game

        # assert it's you turn
        assert(game.turn == self.id_), f"It's not your turn: it's turn of player: {self.game.turn}"
        
        #self.check_input(card_declared)
        
        game.declare_card(card_declared)
        
    def play_cards(self, cards:str):
        """ Input: cards -> list.
            Purpose: remove from hand the cards played, 
            and add them to the pool. 
            Controls that each card played is actually in hand,
            otherwise disregards them
            Note: dovresti spostare tutto questo nella classe GAME"""
        
        game = self.game
        
        # assert you haven't already played cards
        
        # assert the card has already been declared
        assert(game.state == game.states[3]), "You first have to declare a card"
        
        # assert it's you turn
        assert(game.turn == self.id_), f"It's not your turn: it's turn of player: {game.turn}"
        
        #self.check_input(cards)
        
        cards = [int(carta) if int(carta) != game.jolly \
                  else (self.hand.remove(int(carta)), int(game.card_declared))[1]   \
                     if int(carta) in self.hand  \
                     else None \
                 for carta in \
                 list(filter(None, re.split(" |,|;", cards))) ]
                
        # remove each played card from the hand, if present, and keep in cards the cards actually played
        cards = [(self.hand.remove(num),num)[1] for _i, num in enumerate(cards) \
                 if num in self.hand and num != None]
                 
        game.play_cards(cards, self.id_)
        
        return 

    def doubt(self):
        game = self.game     
        doubted_id_ = [i for i in range(1, game.num_players+1)][game.turn - 2]
        
        # assert no self doubt (al turno 1 il giocatore 0 non può dubitare e così via)
        assert(self.id_ != doubted_id_), \
               f"il giocatore {self.id_} non può dubitare di se stesso!"         
        print(f"Il giocatore {self.id_} sta dubitando del giocatore {doubted_id_}!")
     
        loser = game.doubt(self.id_)      
        game.soft_reset(loser)    
        return
        
        
    def check_input(self, inputto, min_=0, max_=13, type_=int, none = False, no_self_doubt = False):
        """ Take an input, while controlling that it is
            within a numerical range or that it is of the
            right type, if the relative arguments are specified """

# se None è una risposta ammissibile e viene data, restituisci None
        if none == True and not inputto:
            return 
        elif none == False and not inputto:
            raise Exception("Input cannot be None")

# here start checks about type, and min < ui < max, and self doubting
        if type_ is not None:
            try:
                inputto = type_(inputto)
            except ValueError:
                raise Exception("Input type must be {0}.".format(type_.__name__))

        if no_self_doubt == True and inputto == self.id_:
            raise Exception("One can't doubt his own play! :/")                
        elif max_ is not None and inputto > max_:
            raise Exception("Input must be less than or equal to {0}.".format(max_))
        elif min_ is not None and inputto < min_:
            raise Exception("Input must be greater than or equal to {0}.".format(min_))

In [58]:
g = Game(2)

p1 = Player(g, 1)
p2 = Player(g, 2)

p1.vote_Start()
p2.vote_Start()

p1.show_hand()

1 players on 2 want to start the game
2 players on 2 want to start the game

instantiating game
pesco le carte iniziali
pesco le carte iniziali
Le tue carte sono:  [1, 3, 3, 3, 3, 4, 5, 5, 5, 6, 7, 7, 7, 8, 8, 8, 9, 10, 10, 11, 11, 11, 12, 13, 13, 13, 13, 100]


In [59]:
p1.declare_card("1")
g.card_declared
p1.show_hand()

La carta da giocare è il 1
Le tue carte sono:  [1, 3, 3, 3, 3, 4, 5, 5, 5, 6, 7, 7, 7, 8, 8, 8, 9, 10, 10, 11, 11, 11, 12, 13, 13, 13, 13, 100]


In [60]:
p1.play_cards("")
p1.show_hand()

1
prima []
dopo []
player 1 has passed its turn
Le tue carte sono:  [1, 3, 3, 3, 3, 4, 5, 5, 5, 6, 7, 7, 7, 8, 8, 8, 9, 10, 10, 11, 11, 11, 12, 13, 13, 13, 13, 100]


In [61]:
p2.doubt()

Il giocatore 2 sta dubitando del giocatore 1!


AssertionError: Non puoi dubitare se il giocatore precedente non ha giocato carte

In [33]:
p2.doubt()

Il giocatore 2 sta dubitando del giocatore 1!
Il giocatore 2 ha correttamente dubitato del giocatore 1!
Le carte sono state prese o rimosse dal pozzo


In [62]:
p2.play_cards("")

1
prima []
dopo []
player 2 has passed its turn
Tutti i giocatori hanno passato! Sta per iniziare un nuovo giro
Le carte sono state prese o rimosse dal pozzo


In [158]:
# dovrebbe dare errore
p1.doubt()

AssertionError: il giocatore 1 non può dubitare di se stesso!

In [159]:
p2.doubt()

Il giocatore 2 sta dubitando del giocatore 1!
E invece il giocatore 1 aveva effettivamente giocato 1 copie della carta 1!
Le carte sono state prese o rimosse dal pozzo


In [160]:
# dovrebbe dare errore
p2.first_draw()

pesco le carte iniziali


AssertionError: Nice try

In [161]:
# dovrebbe dare errore
p2.play_cards([2])

AssertionError: You first have to declare a card

In [162]:
p2.show_hand()
p2.declare_card([2])

Le tue carte sono:  [1, 1, 1, 1, 2, 2, 2, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, 8, 9, 10, 10, 12, 12, 12, 13, 13, 100, 100, 100]
La carta da giocare è il 2


In [163]:
# dovrebbe dare errore
p2.declare_card([2])

AssertionError: A card has already been declared: [2]

In [164]:
p2.play_cards("2, 4; 3")

player 2 has played 3 copies of the card 2 


In [165]:
# dovrebbe dare errore
p2.play_cards("2,")

AssertionError: It's not your turn: it's turn of player: 1

In [166]:
p1.play_cards("2,")


player 1 has played 1 copies of the card 2 


In [167]:
p2.doubt()

Il giocatore 2 sta dubitando del giocatore 1!
E invece il giocatore 1 aveva effettivamente giocato 1 copie della carta 2!
Le carte sono state prese o rimosse dal pozzo


In [168]:
p2.declare_card([2])

La carta da giocare è il 2


In [169]:
p2.play_cards("")

player 2 has passed its turn


In [170]:
p1.play_cards("")

player 1 has passed its turn
Tutti i giocatori hanno passato! Sta per iniziare un nuovo giro
Le carte sono state prese o rimosse dal pozzo


In [171]:
p2.show_hand()

Le tue carte sono:  [1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, 8, 9, 10, 10, 12, 12, 12, 13, 13, 100, 100, 100]


In [173]:
p2.declare_card([1])
p2.play_cards("1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, 8, 9, 10, 10, 12, 12, 12, 13, 13, 100, 100, 100")

La carta da giocare è il 1
Attenzione, il giocatore 2 ha esaurito le carte in mano!
Se nessuno dubita, vincerà appena il giocatore successivo giocherà una carta
player 2 has played 27 copies of the card 1 


In [174]:
p1.play_cards("1")

player 1 has passed its turn


# Game

magari fai in modo che non ci si possa unire a un game già iniziato

cosa succede se uno dubita e uno gioca nello stesso momento?

cambia card declared in stringa o qualcosa di comodo da scrivere\visualizzare

anziché dover passare l'id del giocatore che chiama le funzioni game, c'è un modo per capire quale istanza di player sta chiamando game?

chiama game players da un'altra funzione, così puoi killare tutto quando finisce il game

In [45]:
### import numpy as np
import random as random
import re
import time

class Game:
    def __init__(self, num_players):
        self.Deck = Cards(num_players)
        self.num_players = num_players
        
        # a cosa corrisponde il jolly?
        self.jolly = 100
        
        self.turn = 1       
        self.player = 0
        self.consecutive_passes = 0
        self.cards_played  = []
        
        self.game_started     = False
        self.to_check_victory = False
        self.card_declared    = ""
        
        self.states = ["starting", "drawing", "declaring", "playing"]
        self.state  = self.states[0]
        
        # contains the instances of the players
        self.players = []
        self.voters  = []
        
    
    def add_player(self, player):
        assert(self.state == self.states[0]), "Game has already started"
        assert(player not in self.players), "Player already in tha game"
        
        self.players.append(player)
        
    def remove_player(self):
        # in teoria dovresti poter rimuoveree anche a game inziiato eprò per ora va bene così
        assert(self.state == self.states[0]), "Game has already started"
        #remove also from self.voters
        return
    
    def vote_Start(self, player):
        assert(self.state == self.states[0]), "Game has already started"
        
        self.voters.append(player)
        votes = len(set(self.voters))
        print(f"{votes} players on {len(self.players)} want to start the game")
        if votes >= len(self.players):
            print("\ninstantiating game")
            self.state = self.states[1]
            
            # magari metti qui il first draw con un for loop e mettila come funzione privata
            for player in self.players:
                    player.first_draw()
            self.state = self.states[2]
        return
            
    def first_draw(self):
        assert(self.state == self.states[1]), "Nice try"

        return self.Deck.draw()
    
    def declare_card(self, card_declared:str):
        
        # assert there's need to declare the card
        assert(self.state == self.states[2]), f"A card has already been declared: {self.card_declared}"
        
        self.game_started = True
        self.card_declared = str(card_declared)
        print(f"La carta da giocare è il {self.card_declared}")
        self.state = self.states[3]
        
    def advance_turn(self):
        self.turn += 1
        if self.turn > self.num_players:
            self.turn = 1
            
    def soft_reset(self, loser=None):
        """
        Resetta pool, need_to_declare e il num di pass consecutivi.
        Se specificato il loser, egli pescherà il pozzo.

        Parameters
        ----------
        loser: istanza del player perdente

        Returns
        ----------
        """
        
        # reset all relevant variables
        self.state = self.states[2]
        self.to_check_victory   = False
        self.consecutive_passes = 0
        self.cards_played = []
        
        # eventually add pool to hand of loser and reset pool 
        if loser:
            loser.add_to_hand(self.Deck.pool)
        else:
            self.Deck.reset_pool()
        
        return
    
    
    def check_victory(self):
        """ Se il giocatore corrente non ha carte in mano, 
            dichiaralo come possibile vincitore.
            Se, il possibile vincitore non ha carte in mano alla seconda chiamata,
            dichiaralo vincitore """
        
        # controlla se il giocatore corrente non ha carte
        potential_winner = self.players[self.turn -1]
        if len(potential_winner.hand) == 0 and not self.to_check_victory:
            print(f"Attenzione, il giocatore {potential_winner.id_} ha esaurito le carte in mano!")
            print("Se nessuno dubita, vincerà appena il giocatore successivo giocherà una carta")
            self.to_check_victory = True
        
        # controlla se il giocatore del turno precedente non ha carte
        elif self.to_check_victory:
            potential_winner = self.players[self.turn-2]
            if len(potential_winner.hand) == 0:
                print("\n\n\n")
                print("-----------------------------")
                print(f"Player {self.turn-1} has won! GGs!")
                print("how d fak do i kill this instance within the class?")
                print("-----------------------------")
            else: 
                self.to_check_victory = False
                
    def play_cards(self, cards_played, player_id):
        """ ask the user for the cards he wants to play,
            take only those which he actually has in hand,
            use the Player class method to play the cards """ 
                
        self.cards_played = cards_played
           
        # informative prints and victory check
        if cards_played:
            print(f"player {player_id} has played {len(cards_played)} copies of the card {self.card_declared} ")
            add_to_pool = self.Deck.add_to_pool(cards_played)
            self.consecutive_passes = 0  
        else:
            print(f"player {player_id} has passed its turn")  
            self.consecutive_passes += 1
            self.check_cons_passes()
            
        self.check_victory()
            
        # aggiorna di chi è il turno
        self.advance_turn()
        
        return 
    
    def check_cons_passes(self):
        """ todo: setta il game turn a chi ha iniziato il giro """
        if self.consecutive_passes == self.num_players:
            print("Tutti i giocatori hanno passato! Sta per iniziare un nuovo giro")
            self.soft_reset()
        return
        
    def doubt(self, who_doubted):
        
        assert(self.state == self.states[3]), "Non puoi dubitare di giocate ancora non effettuate!"
        # questo serve, quello dopo è ridondante
        assert(len(self.cards_played)> 0), "Non puoi dubitare se il giocatore precedente non ha giocato carte"
        
        assert(self.consecutive_passes == 0), "Non puoi dubitare se il giocatore precedente non ha giocato carte"
        
        doubted_player = [i for i in range(1, self.num_players + 1)][self.turn-2]
        
# if the player was honest, reset and give the pool to the doubter
        print(f"carte da giocare {[int(self.card_declared)]*len(self.cards_played)}, carte giocate {self.cards_played}")
        if self.cards_played == [int(self.card_declared)]*len(self.cards_played):
            print("E invece il giocatore {} aveva effettivamente giocato {} copie della carta {}!" \
                  .format(doubted_player, len(self.cards_played), self.card_declared)) 
            loser = self.players[who_doubted-1]

# if the player got correctly doubted, give him the first turn and the bluffer takes the cards
        else:
            print(f"Il giocatore {who_doubted} ha correttamente dubitato del giocatore {doubted_player}!")
            loser = self.players[doubted_player-1]
            
        self.soft_reset(loser)
        return
            
        
    def play(self):
        
        self.first_draw()
                
        while True:
            
            """---------------- define player ----------------"""
            
            self.player = self.players[self.turn]
            print("\n----------------------------------------")
            print(f"è il turno del giocatore {self.player.id_}")
            
            
            """---------------- play card ----------------"""
            
            if self.need_to_declare:
                print("Inizia un nuovo giro! Tagadà!")
                time.sleep(1)
                self.player.show_hand()
                # declare the card which will need to be played
                self.card_declared = [self.take_input(istrz=self.decl_istr)]
                self.need_to_declare = False
                
            print(f"La carta da giocare è il {self.card_declared[0]}")
            
            time.sleep(0.5)
            # play cards
            self.cards_played = self.play_cards()         
            
            """---------------- doubt ----------------"""
            
            time.sleep(1)
            who_doubted = None
            # se non è stato passato il turno, apri la fase di dubbio
            if self.cards_played:
                # if someone wants to doubt, insert his player number in this input
                who_doubted = self.take_input(self.doubt_istr, 
                                              min_= 0,
                                              max_=self.num_players,
                                              type_=int,
                                              none = True,
                                              no_self_doubt = True)
                if who_doubted:
                    correctness_doubt = self.doubt(self.cards_played, who_doubted)   
                    time.sleep(2.5)
            
            else: 
                self.consec_passes += 1
                if self.consec_passes >= self.num_players:
                    time.sleep(0.5)
                    print("Tutti i giocatori hanno passato!")
                    
                    self.reset()
            
            """---------------- check victory ----------------"""
            
            if not self.player.hand:
                print(f"\n Player {self.player._id} has won the game, ggs! \n")
                return
            
            """---------------- manage turns ----------------"""
            
            # if there was a doubt, give the turn to the winner of the outcome
            if who_doubted and correctness_doubt == True:
                self.turn = who_doubted-1
                
            # else regularly advance the turn
            elif not who_doubted:
                self.advance_turn()
        
        return